In [14]:
import pandas as pd
from pathlib import Path
from linearmodels.panel import FamaMacBeth

# 1. 数据路径与加载

In [15]:
root_path = Path('../')
data_path = root_path / 'data' / 'final_data_panel.pkl'

data = pd.read_pickle(data_path)

proxys = ['beta_MKT', 'retvol', 'ivol', 'CFVOL', 'age', 'disp']
data

,date,asset,beta_MKT,ivol,retvol,age,disp,CFVOL,LOGBM,LOGME,mom_minus1_0,mom_minus12_minus1,mom_minus36_minus12,mom_minus11_minus2,cgo,turnover,future_return,score
0,2014-01,000006,0.714892,-0.385213,-0.253707,-0.698990,NaN,-0.195242,0.852785,-0.290561,-1.010538,-0.434441,1.405009,0.228706,-0.437893,-0.381935,-0.016018,-1.735858
1,2014-01,000021,0.902618,-0.210387,0.403084,-0.672945,NaN,0.330226,0.302500,0.840230,2.201666,0.015993,-1.109247,0.205026,0.858371,-0.227697,-0.077961,-0.956005
2,2014-01,000028,-0.789488,0.177394,0.003037,-0.682054,NaN,-0.804418,-1.637611,1.817995,1.169076,0.485133,1.219088,0.203879,2.514612,-0.564752,-0.078858,1.029076
3,2014-01,000030,-0.926812,3.139430,3.924975,-0.680567,NaN,0.162950,0.281369,0.279086,-0.336719,-1.182369,0.487879,-1.298161,-0.311534,-0.033620,0.255537,1.454450
4,2014-01,000031,0.503439,-0.779173,-0.700685,-0.679068,NaN,-0.857062,0.960635,-0.096190,-0.409915,-0.677725,-0.352625,-0.694764,-0.952216,-0.911117,-0.090141,-0.388839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57398,2023-12,688772,NaN,NaN,1.188984,2.479040,1.459955,-0.483142,-0.825818,0.268728,1.004084,0.500160,NaN,-0.035467,1.150507,0.641422,-0.390277,0.647214
57399,2023-12,688778,NaN,NaN,0.525038,2.227011,0.561135,0.007475,0.054129,-0.952300,0.288576,-0.910079,NaN,-0.952374,-0.640407,-0.292542,-0.240666,0.714458
57400,2023-12,688779,NaN,NaN,-0.225561,2.227011,1.209956,0.853260,0.108926,-1.478866,0.033653,-1.592488,NaN,-1.541132,-2.424496,0.662592,-0.230137,0.961017
57401,2023-12,688819,-0.840286,NaN,0.298146,1.578404,0.809786,0.458108,0.156839,0.563051,-0.531444,-0.587884,NaN,-0.364451,-0.494163,0.508415,-0.116046,0.198923


# 2. Fama-MacBeth 回归准备

In [16]:
data['date'] = data['date'].dt.to_timestamp('M')
data = data.set_index(['asset', 'date'])
data.index.get_level_values('date').dtype

dtype('<M8[ns]')

In [17]:
data

,,beta_MKT,ivol,retvol,age,disp,CFVOL,LOGBM,LOGME,mom_minus1_0,mom_minus12_minus1,mom_minus36_minus12,mom_minus11_minus2,cgo,turnover,future_return,score
asset,date,,,,,,,,,,,,,,,,
000006,2014-01-31,0.714892,-0.385213,-0.253707,-0.698990,NaN,-0.195242,0.852785,-0.290561,-1.010538,-0.434441,1.405009,0.228706,-0.437893,-0.381935,-0.016018,-1.735858
000021,2014-01-31,0.902618,-0.210387,0.403084,-0.672945,NaN,0.330226,0.302500,0.840230,2.201666,0.015993,-1.109247,0.205026,0.858371,-0.227697,-0.077961,-0.956005
000028,2014-01-31,-0.789488,0.177394,0.003037,-0.682054,NaN,-0.804418,-1.637611,1.817995,1.169076,0.485133,1.219088,0.203879,2.514612,-0.564752,-0.078858,1.029076
000030,2014-01-31,-0.926812,3.139430,3.924975,-0.680567,NaN,0.162950,0.281369,0.279086,-0.336719,-1.182369,0.487879,-1.298161,-0.311534,-0.033620,0.255537,1.454450
000031,2014-01-31,0.503439,-0.779173,-0.700685,-0.679068,NaN,-0.857062,0.960635,-0.096190,-0.409915,-0.677725,-0.352625,-0.694764,-0.952216,-0.911117,-0.090141,-0.388839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688772,2023-12-31,NaN,NaN,1.188984,2.479040,1.459955,-0.483142,-0.825818,0.268728,1.004084,0.500160,NaN,-0.035467,1.150507,0.641422,-0.390277,0.647214
688778,2023-12-31,NaN,NaN,0.525038,2.227011,0.561135,0.007475,0.054129,-0.952300,0.288576,-0.910079,NaN,-0.952374,-0.640407,-0.292542,-0.240666,0.714458
688779,2023-12-31,NaN,NaN,-0.225561,2.227011,1.209956,0.853260,0.108926,-1.478866,0.033653,-1.592488,NaN,-1.541132,-2.424496,0.662592,-0.230137,0.961017


# 3. 设计矩阵构造函数

In [18]:
def prepare_design(data, proxy):
    cols = [
        'future_return', 'cgo', proxy, 'score', 'LOGBM', 'mom_minus1_0', 'LOGME', 'turnover', 'mom_minus12_minus1', 'mom_minus36_minus12'
    ]
    working_df = data[cols].dropna()

    working_df['proxy*cgo'] = working_df[proxy] * working_df['cgo']
    working_df['proxy*score'] = working_df[proxy] * working_df['score']

    y = working_df['future_return']
    X = working_df[[
        'cgo', proxy, 'proxy*cgo', 'score',  'proxy*score',
        'LOGBM', 'LOGME', 'mom_minus1_0', 'mom_minus12_minus1', 'mom_minus36_minus12', 'turnover'
    ]]
    X = X.assign(const=1)
    X = X[['const'] + [c for c in X.columns if c != 'const']]
    return y, X

# 4. 运行 Fama-MacBeth 回归

In [19]:
results = {}
for proxy in proxys:
    y, X = prepare_design(data, proxy)
    mod = FamaMacBeth(y, X)
    res = mod.fit(cov_type='kernel', bandwidth=3)
    results[proxy] = res

# 查看某个 proxy 的结果
print(results['beta_MKT'].summary)

                            FamaMacBeth Estimation Summary                           
Dep. Variable:               future_return   R-squared:                        0.0022
Estimator:                     FamaMacBeth   R-squared (Between):              0.0319
No. Observations:                    53883   R-squared (Within):               0.0024
Date:                     Sat, Dec 13 2025   R-squared (Overall):              0.0022
Time:                             19:40:15   Log-likelihood                 3.604e+04
Cov. Estimator:    Fama-MacBeth Kernel Cov                                           
                                             F-statistic:                      10.881
Entities:                             1187   P-value                           0.0000
Avg Obs:                            45.394   Distribution:                F(11,53871)
Min Obs:                            1.0000                                           
Max Obs:                            120.00   F-statist

In [20]:
for proxy in proxys:
    res = results[proxy]
    coef_df = res.params.reset_index()
    coef_df.columns = ['variable', 'coefficient']
    coef_df['t_stat'] = res.tstats.values
    print(f"Results for proxy: {proxy}")
    print(coef_df)
    print("\n")

Results for proxy: beta_MKT
               variable  coefficient    t_stat
0                 const     0.006741  0.978872
1                   cgo     0.000302  0.284137
2              beta_MKT    -0.000097 -0.109954
3             proxy*cgo     0.000039  0.075921
4                 score    -0.001599 -0.980220
5           proxy*score     0.000408  0.927273
6                 LOGBM    -0.000918 -0.430066
7                 LOGME    -0.002819 -2.680560
8          mom_minus1_0     0.000092  0.067286
9    mom_minus12_minus1     0.001238  0.720255
10  mom_minus36_minus12    -0.001422 -1.555142
11             turnover    -0.004553 -3.771300


Results for proxy: retvol
               variable  coefficient    t_stat
0                 const     0.006785  0.992214
1                   cgo     0.000029  0.027275
2                retvol    -0.000860 -0.997600
3             proxy*cgo    -0.000555 -0.791903
4                 score    -0.001204 -0.736385
5           proxy*score     0.000724  1.463867
6   